## 1. Setup and Authentication

Import required libraries and authenticate to Azure Sentinel.

In [ ]:
# Import required libraries
import os
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.monitor.query import LogsQueryClient, LogsQueryStatus

# Configuration
WORKSPACE_ID = os.getenv("SENTINEL_WORKSPACE_ID", "your-workspace-id-here")

print(f"Workspace ID: {WORKSPACE_ID}")
print(f"Current Time: {datetime.now().isoformat()}")

In [ ]:
# Authenticate to Azure
# Uses Azure CLI credentials by default (run 'az login' first)
try:
    credential = AzureCliCredential()
    client = LogsQueryClient(credential)
    print("✅ Successfully authenticated using Azure CLI")
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    print("\nTip: Run 'az login' in terminal to authenticate")

## 2. Helper Functions

Define reusable functions for querying and data processing.

In [ ]:
def run_kql_query(query: str, timespan: timedelta = None) -> pd.DataFrame:
    """
    Execute a KQL query and return results as a pandas DataFrame.

    Args:
        query: KQL query string
        timespan: Time range for the query (default: None, uses query's time filter)

    Returns:
        pandas DataFrame with query results
    """
    try:
        response = client.query_workspace(workspace_id=WORKSPACE_ID, query=query, timespan=timespan)

        if response.status == LogsQueryStatus.SUCCESS:
            # Convert to DataFrame
            table = response.tables[0]
            df = pd.DataFrame(data=table.rows, columns=[col.name for col in table.columns])
            print(f"✅ Query executed successfully. Rows returned: {len(df)}")
            return df
        else:
            print(f"❌ Query failed with status: {response.status}")
            return pd.DataFrame()

    except Exception as e:
        print(f"❌ Error executing query: {e}")
        return pd.DataFrame()


def display_summary(df: pd.DataFrame, title: str = "Query Results"):
    """
    Display a summary of DataFrame results.
    """
    print(f"\n{'=' * 80}")
    print(f"{title}")
    print(f"{'=' * 80}")
    print(f"Total Records: {len(df)}")
    print(f"Columns: {', '.join(df.columns)}")
    print(f"\nFirst 5 rows:")
    display(df.head())


print("✅ Helper functions defined")

## 3. Hunt: Suspicious RDP Connections

Hunt for RDP connections from unusual sources or during unusual times.

In [ ]:
# Query for suspicious RDP connections
rdp_query = """
let timeframe = 7d;
let knownAdminSources = dynamic(["10.0.1.10", "10.0.1.11"]); // Update with your IPs
SecurityEvent
| where TimeGenerated > ago(timeframe)
| where EventID == 4624 // Successful logon
| where LogonType == 10 // RemoteInteractive (RDP)
| where IpAddress !in (knownAdminSources)
| extend Hour = hourofday(TimeGenerated)
| where Hour < 7 or Hour > 19 // After hours
| summarize 
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated),
    ConnectionCount = count(),
    UniqueTargets = dcount(Computer),
    Targets = make_set(Computer, 10)
    by Account, IpAddress, LogonType
| where ConnectionCount > 2
| order by ConnectionCount desc
"""

print("Executing RDP hunt query...")
rdp_results = run_kql_query(rdp_query)
display_summary(rdp_results, "Suspicious RDP Connections")

In [ ]:
# Visualize RDP connections by source IP
if not rdp_results.empty:
    fig = px.bar(
        rdp_results,
        x="IpAddress",
        y="ConnectionCount",
        color="Account",
        title="After-Hours RDP Connections by Source IP",
        labels={"ConnectionCount": "Number of Connections", "IpAddress": "Source IP Address"},
        hover_data=["UniqueTargets", "FirstSeen", "LastSeen"],
    )
    fig.show()
else:
    print("No suspicious RDP connections found.")

## 4. Hunt: Failed Login Attempts (Brute Force)

Detect potential brute force attacks by analyzing failed login patterns.

In [ ]:
# Query for brute force attempts
brute_force_query = """
let timeWindow = 1h;
SecurityEvent
| where TimeGenerated > ago(24h)
| where EventID == 4625 // Failed logon
| where LogonType in (2, 3, 10) // Interactive, Network, or RDP
| summarize 
    FailedAttempts = count(),
    TargetAccounts = dcount(TargetUserName),
    Accounts = make_set(TargetUserName, 20),
    FirstAttempt = min(TimeGenerated),
    LastAttempt = max(TimeGenerated)
    by IpAddress, Computer
| where TargetAccounts >= 5 // Targeting multiple accounts
| where FailedAttempts >= 10 // Multiple failures
| extend Severity = case(
    FailedAttempts > 50, "Critical",
    FailedAttempts > 25, "High",
    "Medium"
)
| order by FailedAttempts desc
"""

print("Executing brute force hunt query...")
brute_force_results = run_kql_query(brute_force_query)
display_summary(brute_force_results, "Potential Brute Force Attempts")

In [ ]:
# Visualize brute force attempts over time
if not brute_force_results.empty:
    fig = go.Figure()

    for idx, row in brute_force_results.iterrows():
        fig.add_trace(
            go.Scatter(
                x=[row["FirstAttempt"], row["LastAttempt"]],
                y=[row["FailedAttempts"], row["FailedAttempts"]],
                mode="lines+markers",
                name=row["IpAddress"],
                line=dict(width=3),
            )
        )

    fig.update_layout(
        title="Brute Force Attack Timeline",
        xaxis_title="Time",
        yaxis_title="Failed Attempts",
        hovermode="closest",
    )

    fig.show()
else:
    print("No brute force attempts detected.")

## 5. Hunt: Privilege Escalation Attempts

Look for users being added to privileged groups.

In [ ]:
# Query for privilege escalation
priv_esc_query = """
let PrivilegedGroups = dynamic([
    "Domain Admins",
    "Enterprise Admins",
    "Schema Admins",
    "Administrators"
]);
SecurityEvent
| where TimeGenerated > ago(7d)
| where EventID == 4728 or EventID == 4732 or EventID == 4756 // Member added to group
| extend GroupName = TargetUserName
| where GroupName in (PrivilegedGroups)
| extend AddedUser = MemberName, AddedBy = SubjectUserName
| project 
    TimeGenerated,
    AddedUser,
    GroupName,
    AddedBy,
    Computer,
    Activity
| order by TimeGenerated desc
"""

print("Executing privilege escalation hunt query...")
priv_esc_results = run_kql_query(priv_esc_query)
display_summary(priv_esc_results, "Privilege Escalation Events")

## 6. Export Findings

Save hunt results for documentation and further analysis.

In [ ]:
# Export results to CSV files
from pathlib import Path

output_dir = Path("../docs/hunt-results")
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

if not rdp_results.empty:
    rdp_file = output_dir / f"rdp_hunt_{timestamp}.csv"
    rdp_results.to_csv(rdp_file, index=False)
    print(f"✅ RDP results saved to: {rdp_file}")

if not brute_force_results.empty:
    bf_file = output_dir / f"brute_force_hunt_{timestamp}.csv"
    brute_force_results.to_csv(bf_file, index=False)
    print(f"✅ Brute force results saved to: {bf_file}")

if not priv_esc_results.empty:
    pe_file = output_dir / f"priv_esc_hunt_{timestamp}.csv"
    priv_esc_results.to_csv(pe_file, index=False)
    print(f"✅ Privilege escalation results saved to: {pe_file}")

print("\n✅ All results exported successfully!")

## 7. Hunt Summary and Findings

Document key findings and recommended actions.

In [ ]:
# Generate hunt summary
print("=" * 80)
print("THREAT HUNT SUMMARY")
print("=" * 80)
print(f"Hunt Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Hunt Duration: 7 days lookback")
print(f"\nFindings:")
print(f"  - Suspicious RDP Connections: {len(rdp_results)} incidents")
print(f"  - Brute Force Attempts: {len(brute_force_results)} sources")
print(f"  - Privilege Escalation Events: {len(priv_esc_results)} events")

# Recommendations
print(f"\nRecommended Actions:")
if not rdp_results.empty:
    print(f"  ⚠️  Review after-hours RDP connections from unknown sources")
    print(f"  ⚠️  Consider implementing conditional access policies")
if not brute_force_results.empty:
    print(f"  ⚠️  Investigate source IPs for brute force attempts")
    print(f"  ⚠️  Consider blocking IPs or implementing account lockout policies")
if not priv_esc_results.empty:
    print(f"  ⚠️  Verify all privilege escalations are authorized")
    print(f"  ⚠️  Review privileged group membership regularly")

if rdp_results.empty and brute_force_results.empty and priv_esc_results.empty:
    print(f"  ✅ No suspicious activity detected in this hunt")

print("=" * 80)

## Next Steps

1. **Triage findings** - Investigate high-priority alerts
2. **Create incidents** - For confirmed malicious activity
3. **Update detections** - Convert successful hunts to analytics rules
4. **Document IOCs** - Extract indicators for threat intelligence
5. **Share findings** - Brief security team on discoveries

## Additional Hunts to Try

- Credential dumping (LSASS access)
- Persistence mechanisms (scheduled tasks, registry keys)
- Data exfiltration (large outbound transfers)
- Command & Control (beaconing, DNS tunneling)
- Lateral movement (SMB, WMI, PSExec)